In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("/sfs/qumulo/qhome/jtb3sud/PARCv2")
import parc.data.data_em as data
#from parc import misc, metrics, visualization
from parc.model import model_em as model


2024-04-30 11:50:53.691776: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 11:50:58.712401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


# Data pipeline

In [2]:
# Get data and normalization
state_seq_whole, vel_seq_whole = data.clip_raw_data(idx_range = (0,150))
state_seq_norm = data.data_normalization(state_seq_whole,3)
vel_seq_norm = data.data_normalization(vel_seq_whole,2)

Processing 5793831.npy
Processing 5733397.npy
Processing 5733384.npy
Processing 5733386.npy
Processing 5793832.npy
Processing 5733402.npy
Processing 5793835.npy
Processing 5793836.npy
Processing 5733392.npy
Processing 5733388.npy
Processing 5733398.npy
Processing 5733389.npy
Processing 5733401.npy
Processing 5793837.npy
Processing 5793825.npy
Processing 5733385.npy
Processing 5793827.npy
Processing 5733394.npy
Processing 5793834.npy
Processing 5793840.npy
Processing 5793838.npy
Processing 5793826.npy
Processing 5733387.npy
Processing 5793830.npy
Processing 5733390.npy
Processing 5733391.npy
Processing 5793829.npy
Processing 5793839.npy
Processing 5733400.npy
Processing 5733396.npy
Processing 5793833.npy


# Training


### Stage 1: Differentiator training

In [3]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,:3],vel_seq_norm[0][:,:,:,:2]))
dataset_label = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,-3:],vel_seq_norm[0][:,:,:,-2:]))
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)

In [4]:
dataset

<_BatchDataset element_spec=((TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)), (TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)))>

In [5]:
tf.keras.backend.clear_session()
# step size needs to be changed to 1/total number of steps
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "differentiator_training")
parc.differentiator.load_weights('parc2_diff_rk4_tatb.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 100, shuffle = True)

Epoch 1/100


2024-04-29 22:19:58.986912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-04-29 22:20:07.227805: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f88adf88c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-29 22:20:07.227861: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-04-29 22:20:07.325035: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 22:20:08.102946: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


232/232 [==============================] - 273s 958ms/step - total_loss: 185.7712
Epoch 2/100
232/232 [==============================] - 222s 955ms/step - total_loss: 181.2075
Epoch 3/100
232/232 [==============================] - 222s 955ms/step - total_loss: 177.8370
Epoch 4/100
232/232 [==============================] - 222s 955ms/step - total_loss: 176.5101
Epoch 5/100
232/232 [==============================] - 222s 955ms/step - total_loss: 174.7990
Epoch 6/100
232/232 [==============================] - 222s 955ms/step - total_loss: 171.2470
Epoch 7/100
232/232 [==============================] - 222s 955ms/step - total_loss: 169.4326
Epoch 8/100
232/232 [==============================] - 222s 955ms/step - total_loss: 167.6857
Epoch 9/100
232/232 [==============================] - 222s 955ms/step - total_loss: 164.7664
Epoch 10/100
232/232 [==============================] - 222s 955ms/step - total_loss: 163.3846
Epoch 11/100
232/232 [==============================] - 222s 955ms/step

In [6]:
parc.differentiator.save_weights('parc2_diff_rk4_tatb.h5')

### Stage 2: Data-driven integration training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "integrator_training")
parc.integrator.load_weights('parc2_int_rk4_tatb.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 125, shuffle = True)

Epoch 1/125


2024-04-30 11:52:17.496255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-04-30 11:52:20.861035: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55c617b17e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-30 11:52:20.861077: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-04-30 11:52:20.974633: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-30 11:52:21.937055: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


232/232 [==============================] - 154s 498ms/step - total_loss: 686.6307
Epoch 2/125
232/232 [==============================] - 115s 494ms/step - total_loss: 567.7000
Epoch 3/125
232/232 [==============================] - 115s 494ms/step - total_loss: 545.1809
Epoch 4/125
232/232 [==============================] - 115s 494ms/step - total_loss: 523.2788
Epoch 5/125
232/232 [==============================] - 115s 494ms/step - total_loss: 514.4716
Epoch 6/125
232/232 [==============================] - 115s 494ms/step - total_loss: 504.3910
Epoch 7/125
232/232 [==============================] - 115s 494ms/step - total_loss: 494.5680
Epoch 8/125
232/232 [==============================] - 115s 494ms/step - total_loss: 488.8584
Epoch 9/125
232/232 [==============================] - 115s 494ms/step - total_loss: 486.2922
Epoch 10/125
232/232 [==============================] - 115s 494ms/step - total_loss: 484.7857
Epoch 11/125
232/232 [==============================] - 115s 494ms/step

In [ ]:
parc.integrator.save_weights('parc2_int_rk4_tatb.h5')